In [1]:
import numpy as np
import pandas as pd
import matplotlib
import torch
import models.registry
import matplotlib.pyplot as plt
import json

plt.rcParams['text.usetex'] = True
plt.rcParams['font.size'] = 12
plt.rcParams['axes.linewidth'] = 1.5
plt.rcParams["figure.figsize"] = (9,6)

### Magnitude Pruning

In [2]:
mask_one = torch.load("../open_lth_data/lottery_2f4be23b7e4c30fb5fbf086d41cfacb7/replicate_1/level_26/main/mask.pth")
mask_two = torch.load("../open_lth_data/lottery_2f4be23b7e4c30fb5fbf086d41cfacb7/replicate_2/level_26/main/mask.pth")
mask_three = torch.load("../open_lth_data/lottery_2f4be23b7e4c30fb5fbf086d41cfacb7/replicate_3/level_26/main/mask.pth")

In [3]:
mask_layer_names = [k for k, _ in mask_one.items()]

In [4]:
# Doesn't always seem to select the same mask
summed = {}
for layer in mask_layer_names:
    sum_masks = mask_one[layer] + mask_two[layer] + mask_three[layer]
    summed[layer] = sum_masks
    print(f"Layer: {layer} In All: {torch.sum(sum_masks[sum_masks==3])} In Two: {torch.sum(sum_masks[sum_masks==2])} In One: {torch.sum(sum_masks[sum_masks==1])}")

Layer: fc_layers.0.weight In All: 0 In Two: 8 In One: 1021
Layer: fc_layers.1.weight In All: 0 In Two: 12 In One: 799
Layer: fc.weight In All: 30 In Two: 140 In One: 384


In [6]:
summed = {}
for layer in mask_layer_names:
    num_in_layer = torch.numel(mask_one[layer])
    mask_one_percent_remain = 100.0*torch.sum(mask_one[layer])/num_in_layer
    mask_two_percent_remain = 100.0*torch.sum(mask_two[layer])/num_in_layer
    mask_three_percent_remain = 100.0*torch.sum(mask_three[layer])/num_in_layer
    tensor = torch.Tensor([mask_one_percent_remain, mask_two_percent_remain, mask_three_percent_remain])
    std_remaining = torch.std(tensor)
    mean_remaining = torch.mean(tensor)
    print(f"Layer: {layer} Mean % Remaining: {mean_remaining} (+/-){std_remaining}%")

Layer: fc_layers.0.weight Mean % Remaining: 0.1458333283662796 (+/-)0.0014728315873071551%
Layer: fc_layers.1.weight Mean % Remaining: 0.9011111259460449 (+/-)0.01677742227911949%
Layer: fc.weight Mean % Remaining: 18.46666717529297 (+/-)0.6429103016853333%


Final layer is most important

### Fisher Pruning

In [10]:
mask_one = torch.load("../open_lth_data/lottery_a36edb229aed327b8e25e0685c350491/replicate_1/level_26/main/mask.pth")
mask_two = torch.load("../open_lth_data/lottery_a36edb229aed327b8e25e0685c350491/replicate_2/level_26/main/mask.pth")
mask_three = torch.load("../open_lth_data/lottery_a36edb229aed327b8e25e0685c350491/replicate_3/level_26/main/mask.pth")

In [11]:
mask_layer_names = [k for k, _ in mask_one.items()]

In [12]:
# Doesn't seem to always select the same mask
summed = {}
for layer in mask_layer_names:
    sum_masks = mask_one[layer] + mask_two[layer] + mask_three[layer]
    summed[layer] = mask_one[layer] + mask_two[layer] + mask_three[layer]
    print(f"Layer: {layer} In All: {torch.sum(sum_masks[sum_masks==3])} In Two: {torch.sum(sum_masks[sum_masks==2])} In One: {torch.sum(sum_masks[sum_masks==1])}")

Layer: fc.weight In All: 12 In Two: 86 In One: 316
Layer: fc_layers.1.weight In All: 3 In Two: 10 In One: 667
Layer: fc_layers.0.weight In All: 0 In Two: 20 In One: 1280


In [13]:
summed = {}
for layer in mask_layer_names:
    num_in_layer = torch.numel(mask_one[layer])
    mask_one_percent_remain = 100.0*torch.sum(mask_one[layer])/num_in_layer
    mask_two_percent_remain = 100.0*torch.sum(mask_two[layer])/num_in_layer
    mask_three_percent_remain = 100.0*torch.sum(mask_three[layer])/num_in_layer
    tensor = torch.Tensor([mask_one_percent_remain, mask_two_percent_remain, mask_three_percent_remain])
    std_remaining = torch.std(tensor)
    mean_remaining = torch.mean(tensor)
    print(f"Layer: {layer} Mean % Remaining: {mean_remaining} (+/-){std_remaining}%")

Layer: fc.weight Mean % Remaining: 13.800000190734863 (+/-)1.2124358415603638%
Layer: fc_layers.1.weight Mean % Remaining: 0.7555555701255798 (+/-)0.02143031544983387%
Layer: fc_layers.0.weight Mean % Remaining: 0.18424035608768463 (+/-)0.0030757561326026917%


Final layer still most important, but more get pruned from it than with magnitude pruning